In [1]:
import chardet
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from sklearn.metrics import classification_report

import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv("data.csv")

In [3]:
class modelANN(object):
    """
    Simple Neural Network
    """
    def __init__(self, inNum=2, hNum=2, lr=0.01, job_type="class"):
        """
        :param iNum: number of neurons in input layer
        :param hNum: number of neurons in hidden layer
        :param lr: leraning rate
        """
        # inicjalizacja learnig rate'u
        # wielkość ta parametryzuje wielkość korekt wag sieci
        self.lr = lr
        
        # ilość wejść
        self.inNum = inNum
        # ilość wyjść
        self.outNum = 1
        
        #inicjalizacja wag
        #warstwa wejściowa to same wejścia, wszystko leci na ukrytą
        
        # warstwa ukryta, hNum neuronów, po dwa wejścia na każdy neuron
        self.W_h = tf.Variable(tf.random.normal([self.inNum,hNum]))
        self.b_h = tf.Variable(tf.zeros([hNum]))
        
        # warstwa wyjściowa, self.outNum neuron z hNum wejściami
        self.W_out = tf.Variable(tf.random.normal([hNum,self.outNum]))
        self.b_out = tf.Variable(tf.zeros([self.outNum]))

        self.job_type = job_type
        self.outSize = 1

    def predict(self, x):
        """
        :param x: input data, 2D - [batch, features], batch could be None, also could be one row of feature, but also 2D
        """
        # feed forward
        # wejściowa warstwa
        layer_in = x
        # warstwa ukryta
            # mnożymy przez wagi i dodajemy biasy
        layer_hidden = tf.add(tf.matmul(layer_in, self.W_h), self.b_h)
            # przechodzimy z sygnałem przez funkcję aktywacji
        layer_hidden = tf.nn.relu(layer_hidden)
        
        # warstwa wyjściowa
            # mnożymy przez wagi i dodajemy biasy
        layer_out = tf.add(tf.matmul(layer_hidden, self.W_out), self.b_out)
            # przechodzimy z sygnałem przez funkcję aktywacji
            # sigmoid z tego względu, aby otrzymać znormalizowane wyjście w zakresie 0-1
        layer_out = tf.nn.sigmoid(layer_out)
        return layer_out

    def weightsUpdate(self, dW_h, db_h, dW_out, db_out):
        """
        :param dW_h:  hidden layer weights derivative
        :param db_h:  hidden layer bias derivative
        :param dW_out:  out layer weights derivative
        :param db_out:  out layer bias derivative
        """
        # korekta wag warstwy ukrytej
        self.W_h.assign_sub(self.lr * dW_h)
        # korekta biasu warstwy ukrytej
        self.b_h.assign_sub(self.lr * db_h)
        # korekta wag warstwy wyjściowej
        self.W_out.assign_sub(self.lr * dW_out)
        # korekta biasu warstwy wyjściowej
        self.b_out.assign_sub(self.lr * db_out)
    
    def lossFunc(self, y_pred, y_true):
        """
        :param y_pred: predicted data, from model
        :param y_true: true output from model
        """
        # zmiana kształtu referencji na potrzeby oblcizeń macierzowych
        y_true = tf.reshape(y_true, (-1, self.outSize))
        if self.job_type == "class":
            return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_true, y_pred))
        elif self.job_type == "regr":
            return tf.reduce_mean(tf.square(y_pred - y_true))
    
    def fit(self, x, y_true):
        """
        :param x: training data
        :param y_true: connected to x, real output
        """
        # definicja taśmy
        with tf.GradientTape() as t:
            # wyznaczenie błędu
            # tutaj jest uruchomiony cały model i policzony błąd
            # więc pod taśmę wchodzi cały model wraz z funkcją błędu !!!!
            y_pred = self.predict(x)
            current_loss = self.lossFunc(y_pred, y_true)
        # odwijamy taśmę i otrzymujemy pochodne
        dW_h, db_h, dW_out, db_out = t.gradient(current_loss, [self.W_h, self.b_h, self.W_out, self.b_out])
        # korygujemy wagi
        self.weightsUpdate(dW_h, db_h, dW_out, db_out)
        # błąd zwracamy do świata zewnętrznego
        return current_loss

In [4]:
df.isnull().sum()

Age            0
Duration       0
Frequency      0
Location       0
Character      0
Intensity      0
Nausea         0
Vomit          0
Phonophobia    0
Photophobia    0
Visual         0
Sensory        0
Dysphasia      0
Dysarthria     0
Vertigo        0
Tinnitus       0
Hypoacusis     0
Diplopia       0
Defect         0
Ataxia         0
Conscience     0
Paresthesia    0
DPF            0
Type           0
dtype: int64

In [5]:
cat_values = df["Type"].unique()
print(cat_values)
is_migraine = [
    'Typical aura with migraine', 
    'Migraine without aura', 
    'Basilar-type aura', 
    'Sporadic hemiplegic migraine',
    'Familial hemiplegic migraine'
]
no_migraine = ['Other', 'Typical aura without migraine']
df_binary = df

df_binary["Type"] = (df_binary["Type"].isin(is_migraine))
# print(df_binary.head())
# print(df_binary.dtypes)
# df_binary["Type"].value_counts()


['Typical aura with migraine' 'Migraine without aura' 'Basilar-type aura'
 'Sporadic hemiplegic migraine' 'Familial hemiplegic migraine' 'Other'
 'Typical aura without migraine']


In [6]:
Y = df_binary["Type"]
X = df_binary[["Age", "Duration", "Frequency", "Location", "Character", "Intensity", "Nausea", "Vomit", "Phonophobia", "Photophobia", "Visual", "Sensory", "Dysphasia", "Dysarthria", "Vertigo", "Tinnitus", "Hypoacusis", "Diplopia", "Defect", "Ataxia", "Conscience", "Paresthesia", "DPF"]]

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 465)
# y_train = np.asarray(y_train).astype('float32')
# y_test = np.asarray(y_test).astype('float32')
print('Training Data Count: {}'.format(X_train.shape[0]))
print('Testing Data Count: {}'.format(X_test.shape[0]))

Training Data Count: 320
Testing Data Count: 80


In [8]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train, y_train)
# X_train = np.asarray(X_train).astype('float32')
X_test = sc.fit_transform(X_test, y_test)
# X_test = np.asarray(X_test).astype('float32')

In [9]:
learning_rates = [0.001, 0.01, 0.05, 0.1]
epochs = 20
X_train, X_test = X_train.astype('float32'), X_test.astype('float32')

results = []

for lr in learning_rates:
    for hNum in range(2, 11):
        # print("Number of hidden neurons in first layer per input neuron: ", hNum)
        # print("Learning rate: ", lr)
        ann = modelANN(inNum=X_train.shape[1], hNum=hNum, lr=lr)

        for _ in range(epochs):
            curr_loss = ann.fit(X_train, y_train)

        # sprawdzam accuracy modelu na danych testowych (20% z datasetu)
        y_pred = ann.predict(X_test)
        y_pred=(y_pred >= 0.5) # inne threshold warto sprawdzic
        temp_result = accuracy_score(y_true=y_test, y_pred=y_pred)
        results.append((lr, hNum, temp_result))
        # print("Accuracy: ", temp_result)  

best_result = results[0]
for r in results:
    if r[2] > best_result[2]:
        best_result = r
print(best_result)


(0.05, 6, 0.9125)
